In [4]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:   
#%sql duckdb://///home/patrizio115/Documents/dmeyf.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [5]:
%%sql
select 'hola' as col

,col
0,hola


In [6]:
%%sql

create or replace table competencia_02 as 

select 
    *
from read_csv_auto('~/buckets/b1/datasets/competencia_02_crudo.csv.gz')

RuntimeError: (duckdb.IOException) IO Error: No files found that match the pattern "~/buckets/b1/datasets/competencia_02_crudo.csv.gz"
[SQL: create or replace table competencia_02 as 

select 
    *
from read_csv_auto('~/buckets/b1/datasets/competencia_02_crudo.csv.gz')]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [4]:
%%sql 

select distinct foto_mes from competencia_02

UsageError: Cell magic `%%sql` not found.


In [5]:
%sql select count(*) from competencia_02

,count_star()
0,4562810


In [6]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_02
), clientes as (
    select distinct numero_de_cliente from competencia_02
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , 
        case
        WHEN ((mes_0 == 0) or (mes_1 is NULL) or (mes_2 is NULL)) THEN NULL
        WHEN (mes_1 == 0) THEN 'BAJA+1'
        WHEN (mes_2 == 0) THEN 'BAJA+2'
        ELSE 'CONTINUA'
        END as clase_ternaria
    from todo t
    left join competencia_02 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria
from clase_ternaria 
where mes_0 = 1

,Success


In [7]:
%%sql
alter table competencia_02 add column clase_ternaria VARCHAR(10)

,Success


In [8]:
%%sql
update competencia_02
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_02.numero_de_cliente = targets.numero_de_cliente and competencia_02.foto_mes = targets.foto_mes;

,Success


In [9]:
%%sql

select 
    clase_ternaria, 
    count(*) as cant,
from competencia_02
group by clase_ternaria

,clase_ternaria,cant
0,CONTINUA,4195878
1,BAJA+2,18987
2,BAJA+1,18843
3,NaN,329102


In [10]:
%%sql
COPY competencia_02 TO '~/buckets/b1/datasets/competencia_02.csv.gz' (FORMAT CSV, COMPRESSION GZIP, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [13]:
%%sql

SELECT 
    foto_mes, 
    clase_ternaria,
    COUNT(*) as cantidad,
FROM competencia_02
WHERE foto_mes in (202101, 202102, 202103, 202104, 202105)
GROUP BY 1,2
ORDER BY 1,2 ASC

,foto_mes,clase_ternaria,cantidad
0,202101,BAJA+1,632
1,202101,BAJA+2,806
2,202101,CONTINUA,160273
3,202102,BAJA+1,809
4,202102,BAJA+2,1042
5,202102,CONTINUA,160513
6,202103,BAJA+1,1050
7,202103,BAJA+2,963
8,202103,CONTINUA,161311
9,202104,BAJA+1,969
